In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datacompy
import os
import numpy as np

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import tree
from sklearn.tree import plot_tree

# modele + Smote
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from ctgan import CTGAN

In [2]:
df = pd.read_csv("D:/ml/xgboost-main/data/ssh_logs/SSH.csv")
df = df.drop(columns=["user", "ts"])
print(df.columns)
df.head()
df = df.drop_duplicates()
df.shape
y = df["target"]
X = df.drop(columns=["target"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
df_data = df.copy()
dell = pd.concat([X_test, y_test], axis=1)      #polaczenie macierzy X_test oraz y_test
#print(dell.shape)                              #271-55=216
target_num = dell['target'].value_counts()
print(f'Klasyfikator: {target_num}')
df_cleaned = df_data.merge(dell, how='left', indicator=True)            # Dodaje kolumnę '_merge' dla identyfikacji
df_cleaned = df_cleaned[df_cleaned['_merge'] == 'left_only'].drop('_merge', axis=1)

Index(['is_private', 'is_failure', 'is_root', 'is_valid', 'not_valid_count',
       'ip_failure', 'ip_success', 'no_failure', 'first', 'td', 'target'],
      dtype='object')
Klasyfikator: target
0    39
1    16
Name: count, dtype: int64


In [8]:
# concat all datasets
file_path4 = "D:\\ml\\xgboost-main\\data\\processed\\generated_data_test.csv"        #sciezka wraz z nazwa pod jaka wygenerowac plik
#df_cleaned = pd.read_csv()
smote_data = pd.read_csv("D:\\ml\\xgboost-main\\data\\generated\\smote_data.csv")
boarderline_smote_data = pd.read_csv("D:\\ml\\xgboost-main\\data\\generated\\boarderlinesmote_data.csv")
gan_data = pd.read_csv("D:\\ml\\xgboost-main\\data\\generated\\GAN_data.csv")

data1 = pd.concat([df_cleaned, smote_data])
data1['source'] = ['original' if i < len(df_cleaned) else 'smote' for i in range(len(data1))]
data2 = pd.concat([boarderline_smote_data, gan_data])
data2['source'] = ['boarderline' if i < len(boarderline_smote_data) else 'gan' for i in range(len(data2))]
data = pd.concat([data1, data2])
data = data.drop_duplicates()
num_duplicates = data.duplicated().sum()
print(num_duplicates)
print(data)
#data.to_csv(file_path4,index=False)
target_num = data['target'].value_counts()
print(f'Klasyfikatory: {target_num}')
columns_list = data.columns
print('Kolumny:\n',columns_list)

y_under = data['target']
X_under = data.loc[:, ['is_private', 'is_failure', 'is_root', 'is_valid', 'not_valid_count',
       'ip_failure', 'ip_success', 'no_failure', 'first', 'td']]


RUS = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = RUS.fit_resample(X_under, y_under)


print("Rozkład przed undersamplingiem:", np.bincount(y_under))
print("Rozkład po undersamplingu:", np.bincount(y_resampled))
num_duplicates = X_resampled.duplicated().sum()
print(num_duplicates)

#dodanie danych tak zeby bylo wiadomo skad pochodza, potem wylaczenie ostatniej kokumny przed undersamplingiem 
#i sprawdzenie % jaki wychodzi wspolczynnik wykorzystanych danych do najoptymalniejszej nauki

0
     is_private  is_failure  is_root  is_valid  not_valid_count  ip_failure  \
0             1           1        0         1                0           1   
2             1           0        0         1                0           0   
3             1           1        0         1                0           1   
4             1           1        0         1                0           2   
5             1           1        1         1                0           3   
..          ...         ...      ...       ...              ...         ...   
105           1           1        0         1               20          14   
106           1           1        0         1                5          26   
107           0           1        1         0                8          30   
108           1           1        1         0               28           8   
109           1           1        0         0               29          38   

     ip_success  no_failure  first   td  target  

In [9]:
data_cal = pd.concat([X_resampled, y_resampled], axis=1)
print(data_cal.head)
num_duplicates = data_cal.duplicated().sum()
print(num_duplicates)
print(f"Liczba duplikatów w X_resampled: {X_resampled.duplicated().sum()}")


<bound method NDFrame.head of      is_private  is_failure  is_root  is_valid  not_valid_count  ip_failure  \
0             1           1        0         1                0           1   
2             1           0        0         1                0           0   
3             1           1        0         1                0           1   
4             1           1        0         1                0           2   
5             1           1        1         1                0           3   
..          ...         ...      ...       ...              ...         ...   
107           0           1        1         1                0           6   
122           1           1        0         0               14          45   
25            1           1        0         0               23          21   
8             1           1        0         1               26          28   
48            0           1        0         1                0           6   

     ip_success  no_f

In [ ]:
data_cal = pd.merge(data_cal, data[['source']], how='left', left_index=True, right_index=True)      #Dopasowanie kolumn w `data` i `data_cal`
comparison = pd.merge(data_cal, data, how='outer', indicator=True)      #Połączenie zbiorów danych w celu identyfikacji unikalnych wierszy
# Krok 4: Obliczenie rozkładu `source` w oryginalnym zbiorze danych
original_source_distribution = data['source'].value_counts()

# Krok 5: Obliczenie rozkładu `source` w zbiorze treningowym
used_in_training = comparison[comparison['_merge'] == 'both']  # Wiersze obecne w obu zbiorach
training_source_distribution = used_in_training['source'].value_counts()

# Krok 6: Obliczenie procentowego użycia danych do treningu dla każdej wartości `source`
source_percentage = (training_source_distribution / original_source_distribution * 100).fillna(0)

# Krok 7: Wyświetlenie wyników
print("Rozkład `source` w oryginalnym zbiorze danych:")
print(original_source_distribution)
print("\nRozkład `source` w zbiorze treningowym:")
print(training_source_distribution)
print("\nProcent użycia danych do treningu dla każdej wartości `source`:")
print(source_percentage)

""" # Opcjonalne: Zapisanie wyników do pliku CSV
result = pd.DataFrame({
    'Oryginalna liczba': original_source_distribution,
    'Liczba w treningu': training_source_distribution,
    'Procent użycia': source_percentage
})
result.to_csv("analiza_uzycia_source.csv", index=True) """


Rozkład `source` w oryginalnym zbiorze danych:
source
original       216
gan            110
boarderline     99
smote           81
Name: count, dtype: int64

Rozkład `source` w zbiorze treningowym:
source
original       198
gan             53
smote           48
boarderline     48
Name: count, dtype: int64

Procent użycia danych do treningu dla każdej wartości `source`:
source
boarderline    48.484848
gan            48.181818
original       91.666667
smote          59.259259
Name: count, dtype: float64


In [6]:
# Łączenie danych z oznaczeniem, skąd pochodzą wiersze
#df_cal = pd.concat([df_cal, data_duplicate["source"]])
""" df_cal = pd.merge(
    df_cal,
    data_duplicate[['source']],  # Wyciągamy tylko kolumnę 'source' z data_duplicate
    how='inner'
    )

"""
print("Liczba wierszy w data_duplicate:", len(data_duplicate))

#print("Liczba wierszy po merge:", len(df_cal))
#print(df_cal) 
unique_data_cal = pd.merge(data_cal, data_duplicate, how="inner")
#unique_data_cal = unique_data_cal.drop_duplicates()
#print(unique_data_cal['source'].isnull().sum())  # Sprawdzenie braków
unique_data_cal = pd.merge(data_cal, df_cal, how='outer', indicator=True)
unique_data_cal = unique_data_cal.drop_duplicates()

# Filtrowanie wierszy występujących tylko w `data_cal`
print(unique_data_cal)

#print("Liczba wierszy w data_cal:", len())
# Dopasowanie na podstawie indeksów (lub wspólnych kolumn)
data_duplicate_indexed = data_duplicate.set_index(['is_private', 'is_failure', 'is_root', 'target'])

unique_data_cal_indexed = unique_data_cal.set_index(['is_private', 'is_failure', 'is_root', 'target'])

# Aktualizacja kolumny 'source' w unique_data_cal
unique_data_cal_indexed['source'].update(data_duplicate_indexed['source'])

# Reset indeksu, aby zachować oryginalną strukturę
unique_data_cal = unique_data_cal_indexed.reset_index()

print("Zaktualizowany unique_data_cal:")
print(unique_data_cal)

NameError: name 'data_duplicate' is not defined